## Session-Triplet-Dataset

Preparação do dataset de triplet para session

In [1]:
spark

In [31]:
from pyspark.sql.functions import collect_set, collect_list, lit, sum, udf, concat_ws, col, count, abs
from pyspark.sql.functions import explode, posexplode

In [3]:
df = spark.read.parquet("../output/ifood/dataset/indexed_intra_session_train_0.10/")
df.show(1)

+--------------------+--------------+--------------+
|          session_id|merchant_idx_A|merchant_idx_B|
+--------------------+--------------+--------------+
|06aa9f97-65f3-45f...|             0|           196|
+--------------------+--------------+--------------+
only showing top 1 row



In [4]:
df.count()

291163

In [5]:
a = 53
b = 1496
df.filter(df.merchant_idx_A == a).filter(df.merchant_idx_B == b).count()

0

In [7]:
df = spark.read.parquet("../output/ifood/ufg_dataset_all/info_session/")
df = df.orderBy(col('click_timestamp'))
df.show(10)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|201b8efb-bf7a-42f...|eefd77bc-2288-472...|83e5a078-2165-4b8...|2019-06-01 00:00:31|  0|  2019-06-01|
|201b8efb-bf7a-42f...|eefd77bc-2288-472...|17b17a05-c383-45f...|2019-06-01 00:00:31|  0|  2019-06-01|
|97d7c14f-9bcc-4a7...|74246816-23fe-42f...|a5d66080-c059-4c9...|2019-06-01 00:01:10|  0|  2019-06-01|
|3977c2b6-b8cf-40a...|51d04693-71e0-493...|2cf13ea7-63be-423...|2019-06-01 00:01:17|  0|  2019-06-01|
|3977c2b6-b8cf-40a...|51d04693-71e0-493...|2cf13ea7-63be-423...|2019-06-01 00:01:17|  1|  2019-06-01|
|3977c2b6-b8cf-40a...|51d04693-71e0-493...|2cf13ea7-63be-423...|2019-06-01 00:01:17|  0|  2019-06-01|
|a80ed5b5-d51e-46a...|5c196c3b-050b-42a...|92c5c3a6-40be-4a8...|2019-06-01 00:01:2

In [11]:
df.filter(df.session_id == '001347db-8240-4a01-818c-5f255323a6fd').show(10)

+--------------------+--------------------+--------------------+-------------------+---+------------+
|          session_id|          account_id|         merchant_id|    click_timestamp|buy|dt_partition|
+--------------------+--------------------+--------------------+-------------------+---+------------+
|001347db-8240-4a0...|753e6059-9396-4d7...|3fd04924-5c23-448...|2019-07-10 20:06:08|  1|  2019-07-10|
|001347db-8240-4a0...|753e6059-9396-4d7...|3fd04924-5c23-448...|2019-07-10 20:11:06|  1|  2019-07-10|
+--------------------+--------------------+--------------------+-------------------+---+------------+



In [10]:
df_session = df.groupby("session_id").agg(collect_list("merchant_id").alias("merchant_ids"), count("merchant_id")).limit(3)
df_session.take(3)

[Row(session_id='000264c8-c999-4639-823a-232db6ca052d', merchant_ids=['517dece4-ffe1-4abb-af89-2b78ad2bc315', '35339dfa-cf09-4943-ad20-a51900600fdd', '92c5c3a6-40be-4a81-8969-3505b7bf16cc', '11234fa1-593b-4781-8fd5-5a7008065bcf'], count(merchant_id)=4),
 Row(session_id='000e51fa-6215-463c-b316-339501563bb5', merchant_ids=['8110b1bb-e037-4a6e-a1c5-18e95c051ad8', '8110b1bb-e037-4a6e-a1c5-18e95c051ad8', '0de90f03-2e49-44f7-a008-594d01a489e8'], count(merchant_id)=3),
 Row(session_id='001347db-8240-4a01-818c-5f255323a6fd', merchant_ids=['3fd04924-5c23-4486-95b4-bebd6f9462e3', '3fd04924-5c23-4486-95b4-bebd6f9462e3'], count(merchant_id)=2)]

In [19]:
df_session_pos = df_session.select(col('session_id').alias('_session_id'), posexplode(df_session.merchant_ids))
df_session_pos.show()

+--------------------+---+--------------------+
|         _session_id|pos|                 col|
+--------------------+---+--------------------+
|000264c8-c999-463...|  0|517dece4-ffe1-4ab...|
|000264c8-c999-463...|  1|35339dfa-cf09-494...|
|000264c8-c999-463...|  2|92c5c3a6-40be-4a8...|
|000264c8-c999-463...|  3|11234fa1-593b-478...|
|000e51fa-6215-463...|  0|8110b1bb-e037-4a6...|
|000e51fa-6215-463...|  1|8110b1bb-e037-4a6...|
|000e51fa-6215-463...|  2|0de90f03-2e49-44f...|
|001347db-8240-4a0...|  0|3fd04924-5c23-448...|
|001347db-8240-4a0...|  1|3fd04924-5c23-448...|
+--------------------+---+--------------------+



In [25]:
df_interaction = df_session.withColumn("merchant_id_A", explode(df_session.merchant_ids))
df_interaction.show()

+--------------------+--------------------+------------------+--------------------+
|          session_id|        merchant_ids|count(merchant_id)|       merchant_id_A|
+--------------------+--------------------+------------------+--------------------+
|000264c8-c999-463...|[517dece4-ffe1-4a...|                 4|517dece4-ffe1-4ab...|
|000264c8-c999-463...|[517dece4-ffe1-4a...|                 4|35339dfa-cf09-494...|
|000264c8-c999-463...|[517dece4-ffe1-4a...|                 4|92c5c3a6-40be-4a8...|
|000264c8-c999-463...|[517dece4-ffe1-4a...|                 4|11234fa1-593b-478...|
|000e51fa-6215-463...|[8110b1bb-e037-4a...|                 3|8110b1bb-e037-4a6...|
|000e51fa-6215-463...|[8110b1bb-e037-4a...|                 3|8110b1bb-e037-4a6...|
|000e51fa-6215-463...|[8110b1bb-e037-4a...|                 3|0de90f03-2e49-44f...|
|001347db-8240-4a0...|[3fd04924-5c23-44...|                 2|3fd04924-5c23-448...|
|001347db-8240-4a0...|[3fd04924-5c23-44...|                 2|3fd04924-5c23-

In [26]:
df_interaction = df_interaction.join(df_session_pos, 
                    (df_interaction.session_id == df_session_pos._session_id) & (df_interaction.merchant_id_A == df_session_pos.col))\
    .select('session_id', 'merchant_id_A', 'pos', 'merchant_ids')
df_interaction = df_interaction.withColumnRenamed('pos', 'pos_A')
df_interaction.show()

+--------------------+--------------------+-----+--------------------+
|          session_id|       merchant_id_A|pos_A|        merchant_ids|
+--------------------+--------------------+-----+--------------------+
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|[517dece4-ffe1-4a...|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|[517dece4-ffe1-4a...|
|000264c8-c999-463...|92c5c3a6-40be-4a8...|    2|[517dece4-ffe1-4a...|
|000264c8-c999-463...|11234fa1-593b-478...|    3|[517dece4-ffe1-4a...|
|000e51fa-6215-463...|8110b1bb-e037-4a6...|    1|[8110b1bb-e037-4a...|
|000e51fa-6215-463...|8110b1bb-e037-4a6...|    0|[8110b1bb-e037-4a...|
|000e51fa-6215-463...|8110b1bb-e037-4a6...|    1|[8110b1bb-e037-4a...|
|000e51fa-6215-463...|8110b1bb-e037-4a6...|    0|[8110b1bb-e037-4a...|
|000e51fa-6215-463...|0de90f03-2e49-44f...|    2|[8110b1bb-e037-4a...|
|001347db-8240-4a0...|3fd04924-5c23-448...|    1|[3fd04924-5c23-44...|
|001347db-8240-4a0...|3fd04924-5c23-448...|    0|[3fd04924-5c23-44...|
|00134

In [27]:
df_interaction2 = df_interaction.withColumn("merchant_id_B", explode(df_session.merchant_ids))
df_interaction2.show()

+--------------------+--------------------+-----+--------------------+--------------------+
|          session_id|       merchant_id_A|pos_A|        merchant_ids|       merchant_id_B|
+--------------------+--------------------+-----+--------------------+--------------------+
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|[517dece4-ffe1-4a...|517dece4-ffe1-4ab...|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|[517dece4-ffe1-4a...|35339dfa-cf09-494...|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|[517dece4-ffe1-4a...|92c5c3a6-40be-4a8...|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|[517dece4-ffe1-4a...|11234fa1-593b-478...|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|[517dece4-ffe1-4a...|517dece4-ffe1-4ab...|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|[517dece4-ffe1-4a...|35339dfa-cf09-494...|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|[517dece4-ffe1-4a...|92c5c3a6-40be-4a8...|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|[517dece4-ffe1-4a...|11234fa1-5

In [28]:
df_interaction2 = df_interaction2.join(df_session_pos, 
                    (df_interaction2.session_id == df_session_pos._session_id) & (df_interaction2.merchant_id_B == df_session_pos.col))\
    .select('session_id', 'merchant_id_A', 'pos_A', 'merchant_id_B', 'pos')
df_interaction2 = df_interaction2.withColumnRenamed('pos', 'pos_B')

df_interaction2.show()

+--------------------+--------------------+-----+--------------------+-----+
|          session_id|       merchant_id_A|pos_A|       merchant_id_B|pos_B|
+--------------------+--------------------+-----+--------------------+-----+
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|517dece4-ffe1-4ab...|    0|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|35339dfa-cf09-494...|    1|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|92c5c3a6-40be-4a8...|    2|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|11234fa1-593b-478...|    3|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|517dece4-ffe1-4ab...|    0|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|35339dfa-cf09-494...|    1|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|92c5c3a6-40be-4a8...|    2|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|11234fa1-593b-478...|    3|
|000264c8-c999-463...|92c5c3a6-40be-4a8...|    2|517dece4-ffe1-4ab...|    0|
|000264c8-c999-463...|92c5c3a6-40be-4a8...|    2|35339dfa-cf09-494...|    1|

In [32]:
df_interaction2 = df_interaction2.withColumn("relative_pos", abs(df_interaction2.pos_A - df_interaction2.pos_B))
df_interaction2.show()

+--------------------+--------------------+-----+--------------------+-----+------------+
|          session_id|       merchant_id_A|pos_A|       merchant_id_B|pos_B|relative_pos|
+--------------------+--------------------+-----+--------------------+-----+------------+
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|517dece4-ffe1-4ab...|    0|           0|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|35339dfa-cf09-494...|    1|           1|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|92c5c3a6-40be-4a8...|    2|           2|
|000264c8-c999-463...|517dece4-ffe1-4ab...|    0|11234fa1-593b-478...|    3|           3|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|517dece4-ffe1-4ab...|    0|           1|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|35339dfa-cf09-494...|    1|           0|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|92c5c3a6-40be-4a8...|    2|           1|
|000264c8-c999-463...|35339dfa-cf09-494...|    1|11234fa1-593b-478...|    3|           2|
|000264c8-

In [29]:
df_session.take(3)

[Row(session_id='000264c8-c999-4639-823a-232db6ca052d', merchant_ids=['517dece4-ffe1-4abb-af89-2b78ad2bc315', '35339dfa-cf09-4943-ad20-a51900600fdd', '92c5c3a6-40be-4a81-8969-3505b7bf16cc', '11234fa1-593b-4781-8fd5-5a7008065bcf'], count(merchant_id)=4),
 Row(session_id='000e51fa-6215-463c-b316-339501563bb5', merchant_ids=['8110b1bb-e037-4a6e-a1c5-18e95c051ad8', '8110b1bb-e037-4a6e-a1c5-18e95c051ad8', '0de90f03-2e49-44f7-a008-594d01a489e8'], count(merchant_id)=3),
 Row(session_id='001347db-8240-4a01-818c-5f255323a6fd', merchant_ids=['3fd04924-5c23-4486-95b4-bebd6f9462e3', '3fd04924-5c23-4486-95b4-bebd6f9462e3'], count(merchant_id)=2)]

In [10]:
df = df_interaction2.select("session_id", "merchant_id_A", "merchant_id_B").distinct().filter(df_interaction2.merchant_id_A != df_interaction2.merchant_id_B)
df.show(20)

+--------------------+--------------------+--------------------+
|          session_id|       merchant_id_A|       merchant_id_B|
+--------------------+--------------------+--------------------+
|000264c8-c999-463...|35339dfa-cf09-494...|92c5c3a6-40be-4a8...|
|000264c8-c999-463...|35339dfa-cf09-494...|517dece4-ffe1-4ab...|
|000264c8-c999-463...|35339dfa-cf09-494...|11234fa1-593b-478...|
|000264c8-c999-463...|92c5c3a6-40be-4a8...|35339dfa-cf09-494...|
|000264c8-c999-463...|92c5c3a6-40be-4a8...|517dece4-ffe1-4ab...|
|000264c8-c999-463...|92c5c3a6-40be-4a8...|11234fa1-593b-478...|
|000264c8-c999-463...|517dece4-ffe1-4ab...|35339dfa-cf09-494...|
|000264c8-c999-463...|517dece4-ffe1-4ab...|92c5c3a6-40be-4a8...|
|000264c8-c999-463...|517dece4-ffe1-4ab...|11234fa1-593b-478...|
|000264c8-c999-463...|11234fa1-593b-478...|35339dfa-cf09-494...|
|000264c8-c999-463...|11234fa1-593b-478...|92c5c3a6-40be-4a8...|
|000264c8-c999-463...|11234fa1-593b-478...|517dece4-ffe1-4ab...|
|000e51fa-6215-463...|0de

In [11]:
df.count()

14

In [35]:
from typing import List

import luigi
import numpy as np

from recommendation.task.data_preparation.ifood import CreateIntraSessionInteractionDataset

ModuleNotFoundError: No module named 'recommendation'